In [ ]:
import datetime
import pytz
import xbos_services_getter

In [ ]:
# HOST_ADDRESSES FOR ALL MICROSERVICES
# SOON WILL BE REPLACED WITH ONE ADDRESS
# NEED TO ONLY DEFINE THE ADDRESS FOR THE SERVICES YOU USE
DISCOMFORT_HOST_ADDRESS="FILL_ME_IN"
HVAC_CONSUMPTION_HOST_ADDRESS="FILL_ME_IN"
TEMPERATURE_BANDS_HOST_ADDRESS="FILL_ME_IN"
PRICE_HOST_ADDRESS="FILL_ME_IN"
OCCUPANCY_HOST_ADDRESS="FILL_ME_IN"
OUTDOOR_TEMPERATURE_HISTORICAL_HOST_ADDRESS="FILL_ME_IN"
BUILDING_ZONE_NAMES_HOST_ADDRESS="FILL_ME_IN"
INDOOR_DATA_HISTORICAL_HOST_ADDRESS="FILL_ME_IN"
INDOOR_TEMPERATURE_PREDICTION_HOST_ADDRESS="FILL_ME_IN"
METER_DATA_HISTORICAL_HOST_ADDRESS="FILL_ME_IN"
OPTIMIZER_HOST_ADDRESS="FILL_ME_IN"

In [ ]:
# CREATE A STUB TO EACH MICROSERVICE THAT YOU USE
# THIS STUB ONLY GETS CALLED ONCE TO ESTABLISH A CONNECTION TO THE SERVICE
discomfort_stub = xbos_services_getter.get_discomfort_stub(DISCOMFORT_HOST_ADDRESS)
hvac_consumption_stub = xbos_services_getter.get_hvac_consumption_stub(HVAC_CONSUMPTION_HOST_ADDRESS)
temperature_bands_stub = xbos_services_getter.get_temperature_band_stub(TEMPERATURE_BANDS_HOST_ADDRESS)
price_stub = xbos_services_getter.get_price_stub(PRICE_HOST_ADDRESS)
occupancy_stub = xbos_services_getter.get_occupancy_stub(OCCUPANCY_HOST_ADDRESS)
outdoor_temperature_historical_stub = xbos_services_getter.get_outdoor_historic_stub(OUTDOOR_TEMPERATURE_HISTORICAL_HOST_ADDRESS)
building_zone_names_stub = xbos_services_getter.get_building_zone_names_stub(BUILDING_ZONE_NAMES_HOST_ADDRESS)
indoor_data_historical_stub = xbos_services_getter.get_indoor_historic_stub(INDOOR_DATA_HISTORICAL_HOST_ADDRESS)
indoor_temperature_prediction_stub = xbos_services_getter.get_indoor_temperature_prediction_stub(INDOOR_TEMPERATURE_PREDICTION_HOST_ADDRESS)
meter_data_historical_stub = xbos_services_getter.get_meter_data_historical_stub(METER_DATA_HISTORICAL_HOST_ADDRESS)
# optimizer_stub = xbos_services_getter.get_optimizer_stub(OPTIMIZER_HOST_ADDRESS)

In [ ]:
# GET A LIST OF ALL BUILDINGS 
buildings = xbos_services_getter.get_buildings(building_zone_names_stub)
print("List of buildings:",buildings)

# GET ALL ZONES FOR EACH BUILDING
for bldg in buildings:
    zones = xbos_services_getter.get_zones(building_zone_names_stub,bldg)
    print("List of zones in building:",bldg,"are:",zones)
    
# ALTERNATIVELY YOU CAN GET A DICTIONARY OF ALL BUILDINGS AND ZONES WITH ONE CALL
buildings_zones = xbos_services_getter.get_all_buildings_zones(building_zone_names_stub)
print("All buildings and zones:",buildings_zones)

In [ ]:
# EXAMPLE TO RETRIEVE DATA FOR ONE DAY AT ONE HOUR INTERVAL STARTING JAN 1ST 2019
start = pytz.timezone('US/Pacific').localize(datetime.datetime(year=2019, month=1, day=1, hour=0, minute=0))
end = start + datetime.timedelta(days=1)
window = "1h"

In [ ]:
# TODO: DOES NOT WORK "Exception calling application: no such table: view_meter"
# GET HISTORICAL BUILDING METER & GREENBUTTON DATA FOR EACH BUILDING
# MEAN, MAX, MIN, COUNT, SUM and RAW
# start = '2019-01-01T00:00:00Z'
# end = '2019-01-02T00:00:00Z'
electric_point_type = 'Building_Electric_Meter'
greenbutton_point_type = 'Green_Button_Meter'
aggregate = 'MEAN' # MEAN, MAX, MIN, COUNT, SUM and RAW
for bldg in buildings:
    print(bldg)
    print(start,end)
    electric_meter_data = xbos_services_getter.get_meter_data_historical(meter_data_historical_stub,bldg,start,end,electric_point_type,aggregate,window)
    greenbutton_meter_data = xbos_services_getter.get_meter_data_historical(meter_data_historical_stub,bldg,start,end,greenbutton_point_type,aggregate,window)
    print(electric_meter_data)
    print(greenbutton_meter_data)

In [ ]:
# GET HISTORICAL OR FORECAST PRICE FOR EACH BUILDING (UP TO 24 HOURS IN THE FUTURE)
# TODO: DOES NOT WORK address fails to bind
# GET THE TARIFF AND UTILITY FOR A GIVEN BUILDING
for bldg in buildings:
    print(bldg)
    tariff_util = xbos_services_getter.get_tariff_and_utility(price_stub,bldg)
    print(bldg,tariff_util)

# GET THE HISTORICAL PRICE FOR A GIVEN TARIFF AND UTILITY
# TODO return all available tariffs and utilities 
# TODO expose these calls in xbos_services_getter
# xbos_services_getter.get_price_utility_tariff()

# YOU CAN ALSO GET THE PRICE FOR A GIVEN BUILDING DIRECTLY
energy_price_type = 'ENERGY' 
demand_price_type = 'DEMAND'
for bldg in buildings:
    energy_price = xbos_services_getter.get_price(price_stub,bldg,energy_price_type,start,end,window)
    demand_price = xbos_services_getter.get_price(price_stub,bldg,demand_price_type,start,end,window)
    print(bldg)
    print(energy_price)
    print(demand_price)

In [ ]:
# GET HISTORICAL OUTDOOR TEMPERATURE FOR EACH BUILDING
for bldg in buildings:
    historical_outdoor_temperature = xbos_services_getter.get_outdoor_temperature_historic(outdoor_temperature_historical_stub,bldg,start,end,window)
    print(bldg)
    print(historical_outdoor_temperature)


In [ ]:
# GET HISTORICAL HVAC DATA FOR EACH ZONE IN EACH BUILDING 
# INDOOR TEMPERATURE, HVAC STATE, COOLING SETPOINT, HEATING SETPOING
for bldg in buildings:
    zones = xbos_services_getter.get_zones(building_zone_names_stub,bldg)
    for zone in zones:
        actions = xbos_services_getter.get_actions_historic(indoor_data_historical_stub,bldg,zone,start,end,window)
        indoor_temperatures = xbos_services_getter.get_indoor_temperature_historic(indoor_data_historical_stub,bldg,zone,start,end,window)
        #TODO add heating and cooling setpoints to microservice and to xbos-service-getter
#         heating_setpoints = xbos_services_getter.get_indoor_heating_setpoint_historic(indoor_data_historical_stub,bldg,zone,start,end,window)
#         cooling_setpoints = xbos_services_getter.get_indoor_cooling_setpoint_historic(indoor_data_historical_stub,bldg,zone,start,end,window)
        print(bldg,zone)
        print(actions)
        print(indoor_temperatures)
#         print(heating_setpoints)
#         print(cooling_setpoints)

In [ ]:
# GET OCCUPANCY FOR EACH ZONE IN EACH BUILDING
for bldg in buildings:
    zones = xbos_services_getter.get_zones(building_zone_names_stub,bldg)
    for zone in zones:
        occupancy = xbos_services_getter.get_occupancy(occupancy_stub,bldg,zone,start,end,window)
        print(bldg,zone)
        print(occupancy)

In [ ]:
# GET COMFORT BAND AND DO NOT EXCEED BAND FOR EACH ZONE IN EACH BUILDING
for bldg in buildings:
    zones = xbos_services_getter.get_zones(building_zone_names_stub,bldg)
    for zone in zones:
        comfort_band = xbos_services_getter.get_comfortband(temperature_bands_stub,bldg,zone,start,end,window)
        do_not_exceed_band = xbos_services_getter.get_do_not_exceed(temperature_bands_stub,bldg,zone,start,end,window)
        print(bldg,zone)
        print(comfort_band)
        print(do_not_exceed_band)

In [ ]:
# GET A DISCOMFORT METRIC FOR A GIVEN TEMPERATURE AND A GIVEN COMFORT BAND
# TODO why do you require a building name as input??
temperature = 90.0 # Current temperature
temperature_low = 60 # 
temperature_high = 80
occupancy = 0.5 # 0.0 -> 1.0
discomfort = xbos_services_getter.get_discomfort(discomfort_stub,'bldg',temperature,temperature_low,temperature_high,occupancy)
print(discomfort)

In [ ]:
# GET HVAC CONSUMPTION FOR A GIVEN ZONE IN A GIVEN BUILDING
for bldg in buildings:
    zones = xbos_services_getter.get_zones(building_zone_names_stub,bldg)
    for zone in zones:
        hvac_consumption = xbos_services_getter.get_hvac_consumption(hvac_consumption_stub,bldg,zone)
        print(bldg,zone,hvac_consumption)

In [ ]:
# GET INDOOR TEMPERATURE PREDICTION FOR A ZONE IN A BUILDING GIVEN AN ACTION 
# (1-2 STAGE HEATING, 1-2 STAGAE COOLING, DO NOTHING)

action = 0 # 0, 1, 2, 3, 4
for bldg in buildings:
    outdoor_temperatures = xbos_services_getter.get_outdoor_temperature_historic(outdoor_temperature_historical_stub,bldg,start,end,window)
    outdoor_temperature = outdoor_temperatures[1]  
    zones = xbos_services_getter.get_zones(building_zone_names_stub,bldg)
    current_zones_temp = {}
    prev_zones_temp = {}
    for zone in zones:
        indoor_temp = xbos_services_getter.get_indoor_temperature_historic(indoor_data_historical_stub,bldg,zone,start,end,window)
        prev_zones_temp[zone] = indoor_temp[0]
        current_zones_temp[zone]= indoor_temp[1]
    for zone in zones:
        # THESE BUILDINGS DON'T HAVE A THERMAL MODEL
        if(bldg=='csu-dominguez-hills' or bldg=='local-butcher-shop' or bldg=='avenal-recreation-center'):
            continue
        print(bldg,zone)
        temperature_prediction = xbos_services_getter.get_indoor_temperature_prediction(indoor_temperature_prediction_stub,bldg,zone,start,action,current_zones_temp[zone],outdoor_temperature,prev_zones_temp[zone],current_zones_temp)
        print(temperature_prediction)

In [ ]:
# 
# TODO: USE OPTIMIZER with xbos-service getter